# Numeriske løsninger [ikke pensum]

Betrakt transportligningen

$$
u_t + au_x = 0
$$

Vi skal undersøke diverse numeriske metoder, og hvordan de oppfører seg 

## 1. Upwind, downwind, og karakteristikker

La oss nå glemme 'fasiten' over, og undersøke hvordan vi kunne diskretisert den konstant hastighet ligning

$$
u_t + au_x = 0, \quad u(x,0)=g(x),
$$

som har en analytisk løsning $u(t,x)=g(x-at)$. Legg merke til at det betyr at $u(t,x)$ er kun avhengig av initialbetingelse på et punkt, $x-at$.

Vi ønsker å kombinere Eulers metode

$$
u(t_{n+1}) = u(t_n) - k a u_x(t_n) 
$$

Med en tilnærming til $u_x$. Husk fra 6_1 at det finnes tre naturlige måter å tilnærme $\big(u_m(t)\big)_x = u_x(t, x_m)$, nemlig

1. Forlengs differanser

$$
(u_m)_x \approx \frac{u_{m+1}-u_m}{h}
$$


2. Baklengs differanser

$$
(u_m)_x \approx \frac{u_{m}-u_{m-1}}{h}
$$

3. Sentrale differanser
$$
(u_m)_x \approx \frac{u_{m+1}-u_{m-1}}{2h}
$$

Det viser seg at bare en er rimelig for vår ligning, hvis vi bruker Eulers metode i tid! Sjekk det med å kode i feltet under.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# koder forlengs metoden
# a hastighet
# fb randbetingelser på høyre side, en funksjon av tid
# h steglengde i rom
# k steglengde i tid
# t nåværende tid
# x0 nåværende status, altså u(x_m)
# return x1, status på neste tidspunkt, altså u(x_m+1)

def forlengs(a, fb, x0, h, k, t):
    n = x0.size
    x1 = np.zeros(n)
    for i in np.arange(0,n-1):
        x1[i] = (1 + a*k/h)*x0[i] - (a*k/h)*x0[i+1]
    x1[-1] = fb(t)
    return x1

# baklengs metoden
# fa randbetingelser på venstre side, en funksjon av tid

def baklengs(a, fa, x0, h, k, t):
    n = x0.size
    x1 = np.zeros(n)
    for i in np.arange(1,n):
        x1[i] = (1 - a*k/h)*x0[i] + (a*k/h)*x0[i-1]
    x1[0] = fa(t)
    return x1

# sentrale differanser-metoden
# trenger både fa og fb, randbetingelser på henholdsvis høyre og venstre

def sentrale(a, fa, fb, x0, h, k, t):
    n = x0.size
    x1 = np.zeros(n)
    for i in np.arange(1,n-1):
        x1[i] = x0[i] + (a*k/(2*h))*x0[i-1] - (a*k/(2*h))*x0[i+1]
    x1[0] = fa(t)
    x1[-1] = fb(t)
    return x1

# randbetingelser venstre side

def fa(x):
    return 2

# randbetingelser høyre side

def fb(x):
    return 0

# setter hastighet a

hastighet = 1

# setter opp gitteret med punkter i tid og rom
# 200 punkter mellom 0 og 1 i tid
# 50 punkter mellom 1 og -1 i rom

T = 1
n = 200

a = -1
b = 1
m = 50

t = np.linspace(0,T,n)
x = np.linspace(a,b,m)

# beregn k og h

k = t[1]-t[0]
h = x[1]-x[0]

# initialiserer u
u = np.zeros((n,m))

# initialbetingelser - lik 2 for x<0 og 0 for x>0
u[0,:] = 1-np.sign(x)

# sjekker a*k/h
# må være under 1 om baklengs-metoden skal funke
print(hastighet*k/h)

# fjern kommentarer fra metoden du vil bruke

for i in np.arange(1,n):
#    u[i,:] = forlengs(hastighet, fb, u[i-1,:],h,k,t[i])
    u[i,:] = baklengs(hastighet, fa, u[i-1,:],h,k,t[i])
#    u[i,:] = sentrale(hastighet, fa, fb, u[i-1,:],h,k,t[i])    

In [ ]:
fig, ax = plt.subplots()

line, = ax.plot(x, u[0,:])

def animate(i):
    line.set_ydata(u[i,:])
    return line,

ani = animation.FuncAnimation(fig, animate, interval=20, blit=True, save_count=t.size)

from IPython.display import HTML
HTML(ani.to_jshtml())

### a) Hvorfor gikk det galt?

Hvorfor gikk det så galt for to strategier som virker rimelig? La oss tenke på karakteristikkene. Skriv $u^n_m = u(t_n, x_m)$. Metodene er


1. Forlengs
$$
u^{n+1}_m = (1 + \frac{ak}{h}) u^n_m - \frac{ak}{h} u^n_{m+1}
$$

2. Baklengs
$$
u^{n+1}_m = (1 - \frac{ak}{h}) u^n_m + \frac{ak}{h} u^n_{m-1}
$$

3. Sentrale
$$
u^{n+1}_m = u^n_m + \frac{ak}{2h} u^n_{m-1} - \frac{ak}{2h} u^n_{m+1}
$$

Antar for tiden at $a>0$.

I tilfelle 1 ser vi at verdien til $u^{n+1}_m$ er bare avhengig av verdiene $u^n_m$ og $u^n_{m+1}$, altså verdiene til høyre for $x_m$. 

Sporer vi tilbake til $n=0$ ser vi at $u^{n+1}_m$ er avhengig av $g(x), x_m \leq x \leq x_{m+n+1}$. Men den eksakte løsningen $u(x_m, t_{n+1})$ er lik $g(x_m - at_{n+1})$, som er utenfor det området! Det er ingen måte at det kan gå opp.

Om vi velger metode 2 derimot, er $u^{n+1}_m$ er avhengig av $g(x), x_{m-n-1} \leq x \leq x_m$. Vi trenger ikke fortvile så lenge at $x_m - at_{n+1}$ befinner seg i intervallet. Det går opp så lenge $h (n-1) > ak (n+1)$

altså

$$
\frac{ak}{h} < 1
$$

### b) Om $a<0$

Metode 2 kalles for en *upwind* metode om $a>0$. Overbevis deg selv at ting er omvendt hvis $a<0$, dvs da er det kun metode 1 som funker, og som får hete *upwind*. Tegn gjerne et bilde som viser karakterstikkene.

### c) Sentraledifferanser

Det forklarer imidlertidig ikke hvorfor sentraledifferanser ikke funker. Det er langt vanskeligere å komme på en intuitiv forklaring på. 

### d) Randbetingelser

Hva slags randbetingelser trenger vi?

Upwind metode trenger bare randbetingelser på upwind side. For eksempel, for å anvende metode 2 på en ligning med $a>0$, savner vi en metode for å beregne $u_0^{n+1}$, da den trenger $u_{-1}^n$ som er utenfor området. Vi setter altså

$$
u^{n+1}_0 = u(x_0,t_{n+1}) = h_1(t_{n+1}),
$$

hvor $h(t)$ er den oppgitte upwind randbetingelse.

På lik linje ville downwind metode trengte heller en randbetingelse på den andre side, altså 

$$
u^{n+1}_m = u(x_m,t_{n+1}) = h_2(t_{n+1}),
$$

Sentraledifferanser og Lax-Friedrichs krever randbetingelser på begge sider. Det virker å gå imot det som vi stad, nemlig at vi trengte bare randbetingelser på upwind-side. Egentlig er det et problem med disse metoder, men det viser seg at vi kan ofte gjette noe rimelig, f eks vi kan ta 

$$
h_2(t) = h_2 = g(x_m),
$$

hvor vi antar at $u$ er konstant på den downwide side og lik initialverdien der.